In [2]:
from models.decision_tree import DecisionTreeModel
from models.adaBoost import AdaBoostModel
from models.xgboost import XGBoostModel
from models.gradient_boosting import GradientBoostingModel
from models.random_forest import RandomForestModel
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, roc_auc_score
from metrics import cross_validate_with_resampling, Metrics
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler, SMOTE
import pandas as pd
import numpy as np

## Reading Data and Setting up Grid search parameters

In [4]:
data = pd.read_csv('data/Telco-Customer-Churn-encoded-data-FE.csv')
label = pd.read_csv('data/Telco-Customer-Churn-encoded-label.csv')

In [6]:
data.head()

,gender_Male,SeniorCitizen_1,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure,MonthlyCharges,TotalCharges,tenure_group,Monthly/Total_Charges,TotalCharges/tenure
0,0,0,1,0,0,0,1,0,1,0,...,0,0,1,0,1,29.85,29.85,0,1.000000,29.850000
1,1,0,0,0,1,1,0,0,1,0,...,0,0,0,1,34,56.95,1889.50,2,0.030140,55.573529
2,1,0,0,0,1,1,0,0,1,0,...,0,0,0,1,2,53.85,108.15,0,0.497920,54.075000
3,1,0,0,0,0,0,1,0,1,0,...,1,0,0,0,45,42.30,1840.75,3,0.022980,40.905556
4,0,0,0,0,1,1,0,0,0,1,...,0,0,1,0,2,70.70,151.65,0,0.466205,75.825000


In [7]:
label[:5]

,Churn
0,0
1,0
2,1
3,0
4,1


In [11]:
X = data.to_numpy()
y = label.to_numpy().ravel()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
grid_search_parameters = {
    "DecisionTreeModel":{
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': range(3, 10),
    'min_samples_split': range(5, 10),
    'min_impurity_decrease': 10.0**np.arange(-5, 0)}
}

grid_search_parameters["XGBClassifier"] = {
    'max_depth': range(3, 10),  # Similar to DecisionTree
    'learning_rate': 10.0 ** np.arange(-5, 0),  # Equivalent to eta in XGBoost
    'n_estimators': range(50, 200, 50),  # Number of gradient boosted trees
    'subsample': np.arange(0.5, 1.0, 0.1),  # Subsample ratio of the training instances
    'colsample_bytree': np.arange(0.5, 1.0, 0.1)  # Subsample ratio of columns when constructing each tree
}

grid_search_parameters["GradientBoostingClassifier"] = {
    'loss': ['log_loss', 'exponential'],  # Loss function to be optimized
    'learning_rate': 10.0 ** np.arange(-5, 0),
    'n_estimators': range(50, 200, 50),
    'max_depth': range(3, 10),
    'min_samples_split': range(5, 10),
    'min_impurity_decrease': 10.0**np.arange(-5, 0)
}

grid_search_parameters["RandomForestClassifier"] = {
    'n_estimators': range(50, 200, 50),  # Number of trees in the forest
    'criterion': ['gini', 'entropy'],
    'max_depth': range(3, 10),
    'min_samples_split': range(5, 10),
    'min_impurity_decrease': 10.0**np.arange(-5, 0)
}

grid_search_parameters["AdaBoostClassifier"] = {
    'n_estimators': range(50, 200, 50),  # Number of weak learners to train iteratively
    'learning_rate': 10.0 ** np.arange(-5, 0),  # Weight applied to each classifier at each boosting iteration
    'algorithm': ['SAMME', 'SAMME.R']  # Algorithm to use
}



### Decision tree Grid search

In [13]:
DecisionTreeModel_gs = DecisionTreeModel("DecisionTreeModel with grid search")
DT_best_params, DT_recempling = DecisionTreeModel_gs.gs_parameter_tune(X, y, grid_search_parameters['DecisionTreeModel'], max_search=200)


  0%|          | 0/200 [00:00<?, ?it/s]d:\miniconda3\envs\ml-project-env\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "d:\miniconda3\envs\ml-project-env\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")
100%|██████████| 200/200 [02:47<00:00,  1.19it/s]

Model: DecisionTreeModel with grid search
Best Parameters: {'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 9, 'min_impurity_decrease': 0.0001}
Resempling: False
Validation Accuracy: 0.7230170107896974


In [16]:

DT_hyper_model = DecisionTreeModel("DecisionTreeModel with hyper parameters")
DT_hyper_model.hyper_parameter(DT_best_params)
if(DT_recempling):
    DT_hyper_model.fit_with_resampling(X_train, y_train)
else:
    DT_hyper_model.fit(X_train, y_train)

y_pred_hyper = DT_hyper_model.predict(X_test)

Metrics(y_pred_hyper, y_test).f1_score()

0.741670536873865

### XGBoost with Grid Search

In [ ]:
xgb_gs = XGBoostModel("XGBoostModel with grid search")
xgb_best_params, xgb_resempling = xgb_gs.gs_parameter_tune(X, y, grid_search_parameters['XGBClassifier'], max_search=200)

In [ ]:
xgb_hyper_model = XGBoostModel("XGBoostModel with hyper parameters")
xgb_hyper_model.hyper_parameter(xgb_best_params)
if(xgb_resempling):
    xgb_hyper_model.fit_with_resampling(X_train, y_train)
else:
    xgb_hyper_model.fit(X_train, y_train)

y_pred_hyper = xgb_hyper_model.predict(X_test)

Metrics(y_pred_hyper, y_test).f1_score()

/home/uadmin/miniconda3/envs/ml-project-env/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [16:00:34] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "criterion", "min_impurity_decrease", "min_samples_split" } are not used.

  warnings.warn(smsg, UserWarning)


0.7235126320159793

### AdaBoost with Grid Search

In [ ]:
AdaBoostModel_gs = AdaBoostModel("AdaBoostModel with grid search")
ab_best_params, ab_resempling = AdaBoostModel_gs.gs_parameter_tune(X, y, grid_search_parameters['AdaBoostClassifier'], max_search=200)

  0%|          | 0/30 [00:00<?, ?it/s]

/home/uadmin/miniconda3/envs/ml-project-env/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/uadmin/miniconda3/envs/ml-project-env/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/uadmin/miniconda3/envs/ml-project-env/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/home/uadmin/miniconda3/envs/ml-project-env/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is 

Model: AdaBoostModel with grid search
Best Parameters: {'n_estimators': 150, 'learning_rate': 0.1, 'algorithm': 'SAMME.R'}
Resempling: True
Validation Accuracy: 0.7334537629782438


In [ ]:
ab_hyper_model = AdaBoostModel("AdaBoostModel with hyper parameters")
ab_hyper_model.hyper_parameter(ab_best_params)
if(ab_resempling):
    ab_hyper_model.fit_with_resampling(X_train, y_train)
else:
    ab_hyper_model.fit(X_train, y_train)

y_pred_hyper = ab_hyper_model.predict(X_test)

Metrics(y_pred_hyper, y_test).f1_score()

/home/uadmin/miniconda3/envs/ml-project-env/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


0.7435896076820467

### Gradient Boosting with Grid Search

In [ ]:
GradientBoostingModel_gs = GradientBoostingModel("GradientBoostingModel with grid search")
gb_best_params, gb_resempling = GradientBoostingModel_gs.gs_parameter_tune(X, y, grid_search_parameters['GradientBoostingClassifier'], max_search=100)

100%|██████████| 100/100 [45:45<00:00, 27.46s/it]

Model: GradientBoostingModel with grid search
Best Parameters: {'loss': 'log_loss', 'learning_rate': 0.1, 'n_estimators': 50, 'max_depth': 3, 'min_samples_split': 7, 'min_impurity_decrease': 0.01}
Resempling: True
Validation Accuracy: 0.7370372507161347


In [ ]:
gb_hyper_model = GradientBoostingModel("GradientBoostingModel with hyper parameters")
gb_hyper_model.hyper_parameter(gb_best_params)
if(gb_resempling):
    gb_hyper_model.fit_with_resampling(X_train, y_train)
else:
    gb_hyper_model.fit(X_train, y_train)

y_pred_hyper = gb_hyper_model.predict(X_test)

Metrics(y_pred_hyper, y_test).f1_score()

0.7553634785234393

In [ ]:
# gb_hyper_model.save('gb_hyper_model.pkl')

In [ ]:
load_model = GradientBoostingModel()
load_model.load('gb_hyper_model.pkl')

In [ ]:
y_pred_load = load_model.predict(X_test)
Metrics(y_pred_load, y_test).f1_score()


0.7553634785234393

### Random Forest Classifier with Grid Search

In [ ]:
RandomForestModel_gs = RandomForestModel("RandomForestModel with grid search")
rf_best_params, rf_resempling = RandomForestModel_gs.gs_parameter_tune(X, y, grid_search_parameters['RandomForestClassifier'], max_search=200)

In [ ]:
rf_hyper_model = RandomForestModel("RandomForestModel with hyper parameters")
rf_hyper_model.hyper_parameter(rf_best_params)
if(rf_resempling):
    rf_hyper_model.fit_with_resampling(X_train, y_train)
else:
    rf_hyper_model.fit(X_train, y_train)

y_pred_hyper = rf_hyper_model.predict(X_test)

Metrics(y_pred_hyper, y_test).f1_score()

In [ ]:
DecisionTreeModel_gs = DecisionTreeModel("DecisionTreeModel with grid search")
DT_best_params, DT_recempling = DecisionTreeModel_gs.gs_parameter_tune(X, y, grid_search_parameters['DecisionTreeModel'], max_search=200)

In [ ]:
DT_hyper_model = DecisionTreeModel("DecisionTreeModel with hyper parameters")
DT_hyper_model.hyper_parameter(DT_best_params)
if(DT_recempling):
    DT_hyper_model.fit_with_resampling(X_train, y_train)
else:
    DT_hyper_model.fit(X_train, y_train)

y_pred_hyper = DT_hyper_model.predict(X_test)

Metrics(y_pred_hyper, y_test).f1_score()

0.7409847115414075